In [2]:
!pip install datasets sacremoses

In [4]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [5]:
train_ds = load_dataset("stanfordnlp/imdb", split="train")
test_ds = load_dataset("stanfordnlp/imdb", split="test")

In [6]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [10]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    max_len = 400
    texts, labels = [], []
    for row in batch:
        labels.append(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[-2])
        texts.append(torch.LongTensor(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[:-2]))

    texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = torch.LongTensor(labels)

    return texts, labels

* labels  
text를 token id로 변환한 리스트에서 뒤에서 두 번째 토큰만 사용  


* texts  
text를 token id로 변환한 리스트에서 마지막 두 개의 토큰을 제외한 것을 input으로 사용  
pad_sequence로 padding을 넣어줌

In [13]:
train_loader = DataLoader(
    train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn
)

In [15]:
from torch import nn
from math import sqrt

class SelfAttention(nn.Module):
    def __init__(self, input_dim, d_model):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model

        self.wq = nn.Linear(input_dim, d_model)
        self.wk = nn.Linear(input_dim, d_model)
        self.wv = nn.Linear(input_dim, d_model)
        self.dense = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask):
        q, k, v = self.wq(x), self.wk(x), self.wv(x)
        score = torch.matmul(q, k.transpose(-1, -2))
        score = score / sqrt(self.d_model)

        if mask is not None:
            score = score + (mask * -1e9)

        score = self.softmax(score)
        result = torch.matmul(score, v)
        result = self.dense(result)

        return result

* B : batch_size, S : seqeunce length (token 개수), D : d_model  
Query의 차원이 (B, S, D)일 때, B개의 (S, D) 행렬이 있는 것  
Q * K^T를 해주기위해 k.transpose(-1, -2)를 하면 (B, D, S), 즉 B개의 (D, S) 행렬로 변환되어
행렬의 곱셈이 가능함  
그 결과 나온 attention score의 차원은 (B, S, S)

* mask * -1e9  
 서로 길이가 다른 text data의 길이를 맞춰주기위해 더해준 것이 [PAD], padding token  
 attention score를 계산할 때는 이 토큰의 영향을 무시하기위해 score에 아주 작은 값인 mask * -1e9를 더해주는 것

* Softmax를 통과하여 확률변수로 표현할 수 있게 변환한 다음 V 행렬을 곱해줌

* self.dense(result)  
 행렬의 곱셈을 통해 도출된 result를 마지막에 Linear(d_model, d_model)을 거치는 이유는 self-attention의 output을 다시 가공하여 다음 layer에 전달하기 위함

In [18]:
class TransformerLayer(nn.Module):
    def __init__(self, input_dim, d_model, dff):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model
        self.dff = dff

        self.sa = SelfAttention(input_dim, d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff),
            nn.ReLU(),
            nn.Linear(dff, d_model)
        )

    def forward(self, x, mask):
        x = self.sa(x, mask)
        x = self.ffn(x)

        return x

* ffn : feed-forward network
* dff : ffn의 dimension

In [21]:
import numpy as np

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)

max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


* Positional Encoding 계산식에서 pos : position, i : dimensio n  
 2i일 때는 sin으로, 2i + 1 일 때는 cos으로 계산  
 angle_rads에서 짝수일 때는 sin, 홀수일 때는 cos을 적용  

* np.arange  
 연속되 숫자 배열을 생성하는데, range()와 비슷하지만 NumPy 배열로 반환됨  

* np.arange(position)[:, None] 의 차원은 (position, 1)  
 np.arange(d_model)[None, :] 의 차원은 (1, d_model)

In [24]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, dff):
        super().__init__()

        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.dff = dff

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
        self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
        self.classification = nn.Linear(d_model, vocab_size) # 출력 차원을 vocab_size로 수정

    def forward(self, x):
        mask = (x == tokenizer.pad_token_id)
        mask = mask[:, None, :]
        seq_len = x.shape[1]

        x = self.embedding(x)
        x = x * sqrt(self.d_model)
        x = x + self.pos_encoding[:, :seq_len]

        for layer in self.layers:
            x = layer(x, mask)

        x = x[:, -1] # 마지막 단어를 예측
        x = self.classification(x)

        return x

In [26]:
model = TextClassifier(len(tokenizer), 32, 2, 32)

In [28]:
print(torch.__version__)
print(torch.backends.mps.is_built())
print(torch.backends.mps.is_available())

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

2.6.0
True
True


In [30]:
from torch.optim import Adam

lr = 0.001
model = model.to(device)
loss_fn = nn.CrossEntropyLoss() # loss_fn 변경 : 다중 분류에 적합한 CrossEntropyLoss

optimizer = Adam(model.parameters(), lr=lr)

In [32]:
def accuracy(model, dataloader):
  correct = 0
  total = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = model(inputs) # 출력의 차원은 (B, vocab_size)
    preds = torch.argmax(outputs, dim=-1) # 마지막 차원에서 가장 큰 값을 고름
      
    correct += (labels == preds).sum().item()
    total += labels.size(0)

  return correct / total

In [34]:
import matplotlib.pyplot as plt

def plot_acc(train_accs, test_accs, label1='train', label2='test'):
  x = np.arange(len(train_accs))

  plt.plot(x, train_accs, label=label1, marker='.')
  plt.plot(x, test_accs, label=label2, marker='v')
  plt.legend()
  plt.show()

In [36]:
train_accs = []
test_accs = []

n_epochs = 50

for epoch in range(n_epochs):
    total_loss = 0.
    model.train()

    for data in train_loader:
        model.zero_grad()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        preds = model(inputs)
        loss = loss_fn(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch + 1} | Train Loss: {total_loss}')

    with torch.no_grad():
        model.eval()
        train_acc = accuracy(model, train_loader)
        test_acc = accuracy(model, test_loader)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
        print(f'=====> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}')

Epoch 1 | Train Loss: 1437.9056161642075
=====> Train acc: 0.560 | Test acc: 0.564
Epoch 2 | Train Loss: 1076.3102086782455
=====> Train acc: 0.560 | Test acc: 0.563
Epoch 3 | Train Loss: 1040.3834310770035
=====> Train acc: 0.561 | Test acc: 0.563
Epoch 4 | Train Loss: 1027.7245573997498
=====> Train acc: 0.561 | Test acc: 0.563
Epoch 5 | Train Loss: 1019.9059237241745
=====> Train acc: 0.561 | Test acc: 0.563
Epoch 6 | Train Loss: 1015.2037024497986
=====> Train acc: 0.561 | Test acc: 0.563
Epoch 7 | Train Loss: 1011.4889980554581
=====> Train acc: 0.561 | Test acc: 0.563
Epoch 8 | Train Loss: 1007.5133594274521
=====> Train acc: 0.561 | Test acc: 0.563
Epoch 9 | Train Loss: 999.297186255455
=====> Train acc: 0.564 | Test acc: 0.566
Epoch 10 | Train Loss: 989.0660314559937
=====> Train acc: 0.563 | Test acc: 0.563
Epoch 11 | Train Loss: 980.3947664499283
=====> Train acc: 0.562 | Test acc: 0.563
Epoch 12 | Train Loss: 971.3178050518036
=====> Train acc: 0.568 | Test acc: 0.570
Epoch 